# SNMP - Simple Network Management Protocol

## Install snmpd tools, agent and mibs

In [ ]:
%%bash

# Install SNMP - Linux
sudo apt update
sudo apt install -y snmp snmpd snmp-mibs-downloader

## Configure agent configuration file (/etc/snmp/snmpd.conf) and snmp tools configuration file (/etc/snmp/snmp.conf)

In [ ]:
%%bash

sudo mv /etc/snmp/snmpd.conf /etc/snmp/snmpd.conf.old

sudo tee /etc/snmp/snmpd.conf > /dev/null <<EOF
rocommunity public
rwcommunity private
EOF

sudo sed -i '/mibs :/s/^/#/' /etc/snmp/snmp.conf

## Start snmpd daemon (agent)

In [ ]:
%%bash

sudo service snmpd restart
sudo service snmpd status

# Mac
# sudo launchctl unload /System/Library/LaunchDaemons/org.net-snmp.snmpd.plist
# sudo launchctl load -w /System/Library/LaunchDaemons/org.net-snmp.snmpd.plist

## List all downloaded mibs

In [ ]:
%%bash

ls -R /usr/share/snmp/mibs

## Show RFC1213 object tree

In [ ]:
%%bash

# Variables
MIB="/usr/share/snmp/mibs/ietf/RFC1213-MIB"

snmptranslate -m $MIB -Tp

## Show the description of a specific object

In [ ]:
%%bash

# Variables
OBJECT="sysDescr"
MIB="/usr/share/snmp/mibs/ietf/RFC1213-MIB"

# Command to extract text from search_string to } excluding the }
sed -n "/$OBJECT OBJECT-TYPE/,/}/p" $MIB

## Using snmpget

In [ ]:
%%bash

# snmpget -v <SNMP version> -c <community> <host> <obj1> <obj2> <obj3> 

snmpget -v 1 -c public localhost sysDescr.0 sysContact.0

## Using snmpset

In [ ]:
%%bash

snmpset -v 1 -c private localhost sysContact.0 s admin@localhost
snmpget -v 1 -c public localhost sysContact.0

## Using snmpgetnext

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost system interfaces

## Using snmpwalk

In [ ]:
%%bash

# run on terminal: sudo tcpdump -i lo -n port 161 -v
snmpwalk -v 1 -c public localhost system

## Other commands

In [ ]:
%%bash

snmptable -v 1 -c public localhost ifTable

In [ ]:
%%bash

snmpdelta -v 1 -c public -Cs -CT localhost sysUpTime.0

In [ ]:
%%bash

killall snmpdelta

## Exercícios - SNMPv1

a) obter o endereço físico (MAC) da 2a. interface de rede da tabela ifTable

In [ ]:
%%bash

snmpget -v 1 -c public localhost ifPhysAddress.2

b) obter o número de mensagens ICMP enviadas e recebidas pelo elemento gerenciado

In [ ]:
%%bash

snmpget -v 1 -c public localhost icmpInMsgs.0 icmpOutMsgs.0

c) obter o número de requisições GET recebidos pelo agente no elemento gerenciado.

In [ ]:
%%bash

snmpget -v 1 -c public localhost snmpInGetRequests.0

d) modificar o nome do elemento gerenciado para "Gerencia"

In [ ]:
%%bash

snmpset -v 1 -c private localhost sysName.0 s "Gerencia"

e) modificar a identificação da localização do elemento gerenciado para "LabRedes"

In [ ]:
%%bash

snmpset -v 1 -c private localhost sysLocation.0 s "LabRedes"

f) obter a descrição da interface localizada na 1a. linha da tabela ifTable

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost ifDescr

g) obter o endereço IP, índice da interface na tabela ifTable, máscara de rede e endereço de broadcast da primeira entrada da tabela ipAddrEntry

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost ipAdEntIfIndex ipAdEntNetMask ipAdEntBcastAddr

h) obter o endereço local e porta local da primeira linha da tabela udpTable

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost udpLocalPort udpLocalAddress

## Using snmpbulkget

In [ ]:
%%bash

snmpbulkget -v2c -c public localhost -Cn2 -Cr3 system interfaces ifIndex ifDescr ifSpeed

In [ ]:
%%bash

snmpbulkget -v2c -c public localhost -Cn4 -Cr5 tcpInSegs tcpOutSegs tcpRetransSegs tcpInErrs tcpConnState tcpConnLocalAddress tcpConnLocalPort tcpConnRemAddress tcpConnRemPort

In [ ]:
%%bash

snmpbulkget -v2c -c public localhost -Cn0 -Cr4 udpLocalAddress udpLocalPort

## Comparing snmpwalk vs snmpbulkwalk



In [ ]:
%%bash

# run on terminal: sudo tcpdump -i lo -n port 161 -v
# snmpwalk -v 1 -c public localhost system

snmpbulkwalk -v 2c -c public localhost system

# Example using pass directive

## snmpd.conf

In [5]:
%%bash

# Add pass directive to snmpd.conf
sudo tee /etc/snmp/snmpd.conf > /dev/null <<EOF
rocommunity public
rwcommunity private

pass .1.3.6.1.3.1234.1 /usr/bin/python3 /tmp/agent.py
EOF

# Restart agent - Linux
sudo service snmpd restart
sudo service snmpd status

# Restart agent - Mac
# sudo launchctl unload /System/Library/LaunchDaemons/org.net-snmp.snmpd.plist
# sudo launchctl load -w /System/Library/LaunchDaemons/org.net-snmp.snmpd.plist

 * snmpd is running


In [ ]:
%%writefile /tmp/objects.json
{
    ".1.3.6.1.3.1234.1.1.0": "Tesla Model S 2024",
    ".1.3.6.1.3.1234.1.2.0": "2024-02-29 14:30:58",
    ".1.3.6.1.3.1234.1.3.0": 23,
    ".1.3.6.1.3.1234.1.4.0": 86,
    ".1.3.6.1.3.1234.1.5.0": 670,
    ".1.3.6.1.3.1234.1.6.0": 267,
    ".1.3.6.1.3.1234.1.7.0": 2,
    ".1.3.6.1.3.1234.1.8.1.1.1": 1,
    ".1.3.6.1.3.1234.1.8.1.1.2": 2,
    ".1.3.6.1.3.1234.1.8.1.1.3": 3,
    ".1.3.6.1.3.1234.1.8.1.1.4": 4,
    ".1.3.6.1.3.1234.1.8.1.2.1": 1,
    ".1.3.6.1.3.1234.1.8.1.2.2": 2,
    ".1.3.6.1.3.1234.1.8.1.2.3": 2,
    ".1.3.6.1.3.1234.1.8.1.2.4": 1,
    ".1.3.6.1.3.1234.1.8.1.3.1": 3,
    ".1.3.6.1.3.1234.1.8.1.3.2": 2,
    ".1.3.6.1.3.1234.1.8.1.3.3": 1,
    ".1.3.6.1.3.1234.1.8.1.3.4": 3
}

In [20]:
%%writefile /tmp/objects.json
{
    "tModelName": "Tesla Model S 2024",
    "tManDate": "2024-02-29 14:30:58",
    "tACTemp": 23,
    "tBatteryPercentage": 86,
    "tHP": 670,
    "tMaxSpeed": 267,
    "tInterLigths": 2,
    "tDoorIndex1": 1,
    "tDoorIndex2": 2,
    "tDoorIndex3": 3,
    "tDoorIndex4": 4,
    "tDoorStatus1": 1,
    "tDoorStatus2": 2,
    "tDoorStatus3": 2,
    "tDoorStatus4": 1,
    "tDoorWindow1": 3,
    "tDoorWindow2": 2,
    "tDoorWindow3": 1,
    "tDoorWindow4": 3
}

Writing /tmp/objects.json


## Agent code

In [23]:
%%writefile /tmp/agent.py

#!/usr/bin/env python3

import sys
import datetime
import socket
import os
import json

oids2objects = {
    ".1.3.6.1.3.1234.1.1.0": "tModelName",
    ".1.3.6.1.3.1234.1.2.0": "tManDate",
    ".1.3.6.1.3.1234.1.3.0": "tACTemp",
    ".1.3.6.1.3.1234.1.4.0": "tBatteryPercentage",
    ".1.3.6.1.3.1234.1.5.0": "tHP",
    ".1.3.6.1.3.1234.1.6.0": "tMaxSpeed",
    ".1.3.6.1.3.1234.1.7.0": "tInterLigths",
    ".1.3.6.1.3.1234.1.8.1.1.1": "tDoorIndex1",
    ".1.3.6.1.3.1234.1.8.1.1.2": "tDoorIndex2",
    ".1.3.6.1.3.1234.1.8.1.1.3": "tDoorIndex3",
    ".1.3.6.1.3.1234.1.8.1.1.4": "tDoorIndex4",
    ".1.3.6.1.3.1234.1.8.1.2.1": "tDoorStatus1",
    ".1.3.6.1.3.1234.1.8.1.2.2": "tDoorStatus2",
    ".1.3.6.1.3.1234.1.8.1.2.3": "tDoorStatus3",
    ".1.3.6.1.3.1234.1.8.1.2.4": "tDoorStatus4",
    ".1.3.6.1.3.1234.1.8.1.3.1": "tDoorWindow1",
    ".1.3.6.1.3.1234.1.8.1.3.2": "tDoorWindow2",
    ".1.3.6.1.3.1234.1.8.1.3.3": "tDoorWindow3",
    ".1.3.6.1.3.1234.1.8.1.3.4": "tDoorWindow4"
}

settable_objects =  {   
    ".1.3.6.1.3.1234.1.3.0": "tACTemp",
    ".1.3.6.1.3.1234.1.7.0": "tInterLigths",
    ".1.3.6.1.3.1234.1.8.1.2.1": "tDoorStatus1",
    ".1.3.6.1.3.1234.1.8.1.2.2": "tDoorStatus2",
    ".1.3.6.1.3.1234.1.8.1.2.3": "tDoorStatus3",
    ".1.3.6.1.3.1234.1.8.1.2.4": "tDoorStatus4",
}

def get_snmp_type(var) -> str:
    if type(var) == str:
        return "string"
    elif type(var) == int:
        return "integer"
    else:
        return "<unknown>"
    
def load_objects(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            return json.load(file)
    else:
        raise FileNotFoundError(f"Arquivo {file_path} não encontrado.")

def save_objects(file_path, objects):
    with open(file_path, 'w') as file:
        json.dump(objects, file)

def get_object(objects, oid):
    field = oids2objects[oid]
    value = objects[field]
    return value

def set_object(objects, oid, value):
    with open("/tmp/objects.json", "w") as file:
        field = settable_objects[oid]
        objects[field] = value
        json.dump(objects, file)

def main():


    file_path = "/tmp/objects.json"
    try:
        objects = load_objects(file_path)
    except FileNotFoundError as e:
        print(str(e))
        return

    #with open("/tmp/agent.log", 'a') as file:
        #file.write(' '.join(sys.argv) + '\n')

    if len(sys.argv) < 3:
        print("Usage: agent.py <request-type> <MIB-oid> [type] [<new-value>]")
        return

    request_type = sys.argv[1]
    oid = sys.argv[2]

    if request_type == "-g":  # GET request
        if oid in oids2objects:
            obj = get_object(objects, oid)
            print(oid)
            print(get_snmp_type(obj))
            print(obj)
        else:
            print("NONE")
            
    elif request_type == "-s":  # SET request
        if oid in settable_objects and len(sys.argv) == 5:
            content_type = sys.argv[3]
            new_content = sys.argv[4]
            if content_type == "i":
                new_content = int(new_content)
            elif content_type == "s":
                new_content = str(new_content)
            set_object(objects, oid, new_content)
            print(oid)
            print(get_snmp_type(new_content))
            print(new_content)
        elif oid in oids2objects:
            obj = get_object(objects, oid)
            print(oid)
            print(get_snmp_type(obj))
            print(obj)
        else:
            print("NONE")

    elif request_type == "-n": #GETNEXT request
        keys = list(oids2objects.keys())
        if oid in oids2objects:
            idx = keys.index(oid) + 1
            if idx < len(keys):
                next_oid = keys[idx]
                obj = get_object(objects, next_oid)
                print(next_oid)
                print(get_snmp_type(obj))
                print(obj)
            else:
                print("NONE")
        else:
            found = False
            for key in keys:
                if oid in key:
                    found = True
                    obj = get_object(objects, key)
                    print(key)
                    print(get_snmp_type(obj))
                    print(obj)
                    break
            if not found:
                print("NONE")
        

if __name__ == "__main__":
    main()

Overwriting /tmp/agent.py


## Testing

In [24]:
%%bash

python3 /tmp/agent.py -g .1.3.6.1.3.1234.1.1.0
python3 /tmp/agent.py -s .1.3.6.1.3.1234.1.3.0 i 25
python3 /tmp/agent.py -n .1.3.6.1.3.1234.1.8.1.1.2


.1.3.6.1.3.1234.1.1.0
string
Tesla Model S 2024
.1.3.6.1.3.1234.1.3.0
integer
25
.1.3.6.1.3.1234.1.8.1.1.3
integer
3


In [3]:
%%bash

snmpset -v2c -c private localhost .1.3.6.1.3.1234.1.3.0 s teste

Error while terminating subprocess (pid=14153): 


Timeout: No Response from localhost


In [42]:
%%writefile MYMIB.txt
MIB-T2 DEFINITIONS ::= BEGIN

IMPORTS
    OBJECT-GROUP FROM SNMPv2-CONF
    OBJECT-TYPE, experimental FROM SNMPv2-SMI
    DisplayString FROM SNMPv2-TC;

teslaMIB MODULE-IDENTITY
    LAST-UPDATED "202406290000Z"
    ORGANIZATION "PUCRS"
    CONTACT-INFO "admin@localhost"
    DESCRIPTION "MIB for acessing information about tesla cars."
::= { experimental 1234 }

tInfo OBJECT IDENTIFIER ::= { teslaMIB 1 }

tModelName OBJECT-TYPE
    SYNTAX  DisplayString (SIZE (0..255))
    MAX-ACCESS  read-only
    STATUS  current
    DESCRIPTION
            "Tesla model name."
    ::= { tInfo 1 }

tManDate OBJECT-TYPE
    SYNTAX  DisplayString (SIZE (0..255))
    MAX-ACCESS  read-only
    STATUS  current
    DESCRIPTION
            "Tesla manufacturing date"
    ::= { tInfo 2 }

tACTemp OBJECT-TYPE
    SYNTAX  INTEGER
    MAX-ACCESS  read-write
    STATUS  current
    DESCRIPTION
            "Temperature set on the air conditioner on celsius."
    ::= { tInfo 3 }

tBatteryPercentage OBJECT-TYPE
    SYNTAX  INTEGER (0..100)
    MAX-ACCESS  read-only
    STATUS  current
    DESCRIPTION
            "Tesla battery percentage"
    ::= { tInfo 4 }

tHP OBJECT-TYPE
    SYNTAX  INTEGER
    MAX-ACCESS  read-only
    STATUS  current
    DESCRIPTION
            "Tesla HP"
    ::= { tInfo 5 }

tMaxSpeed OBJECT-TYPE
    SYNTAX  INTEGER
    MAX-ACCESS  read-only
    STATUS  current
    DESCRIPTION
            "Tesla max speed reached"
    ::= { tInfo 6 }

tInterLigths OBJECT-TYPE
    SYNTAX  INTEGER {
                    on(1),
                    off(2),
                }
    MAX-ACCESS  read-write
    STATUS  current
    DESCRIPTION
            "Tesla interior lights config"
    ::= { tInfo 7 }

tDoorTable OBJECT-TYPE 
    SYNTAX SEQUENCE OF tDoorEntry
    MAX-ACCESS not-accessible
    STATUS current
    DESCRIPTION
    "A list of door info entries." 
    ::= { tInfo 8 }

tDoorEntry OBJECT-TYPE
    SYNTAX  tDoorEntry
    MAX-ACCESS  not-accessible
    STATUS  current
    DESCRIPTION "Information entry about vehicle doors."
    INDEX   { tDoorIndex }
    ::= { tDoorTable 1 }

tDoorEntry ::=
    SEQUENCE {
        tDoorStatus    INTEGER,
        tDoorWindow    INTEGER
    }

tDoorIndex OBJECT-TYPE
    SYNTAX  INTEGER { front-right(1), front-left(2), rear-right(3), rear-left(4) }
    MAX-ACCESS  read-only
    STATUS  current
    DESCRIPTION "Door identifier."
    ::= { tDoorEntry 1 }

tDoorStatus OBJECT-TYPE
    SYNTAX  INTEGER { open(1), closed(2) }
    MAX-ACCESS  read-write
    STATUS  current
    DESCRIPTION "Door status, tracks opening and closing of doors."
    ::= { tDoorEntry 2 }

tDoorWindow OBJECT-TYPE
    SYNTAX  INTEGER { open(1), half-open(2), closed(3) }
    MAX-ACCESS  read-only
    STATUS  current
    DESCRIPTION "Door window status."
    ::= { tDoorEntry 3 }

END

Overwriting MYMIB.txt


In [43]:
%%bash

snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.1.0
snmpget -v2c -c public -M +. -m +MIB-T2 localhost .1.3.6.1.3.1234.1.1.0
snmpget -v2c -c public -M +. -m +MIB-T2 localhost tMaxSpeed.0
snmpget -v2c -c public -M +. -m +MIB-T2 localhost tInterLigths.0
snmpget -v2c -c private -M +. -m +MIB-T2 localhost tACTemp.0


SNMPv2-SMI::experimental.1234.1.1.0 = STRING: "Tesla Model S 2024"
MIB-T2::tModelName.0 = STRING: Tesla Model S 2024
MIB-T2::tMaxSpeed.0 = INTEGER: 267
MIB-T2::tInterLigths.0 = INTEGER: off(2)


MIB-T2::tACTemp.0 = INTEGER: 25


In [26]:
%%bash

snmpset -v2c -c private -M +. -m +MIB-T2 localhost tInterLigths.0 i 1
snmpget -v2c -c public -M +. -m +MIB-T2 localhost tInterLigths.0

snmpset -v2c -c private -M +. -m +MIB-T2 localhost tACTemp.0 i 25
snmpget -v2c -c private -M +. -m +MIB-T2 localhost tACTemp.0

MIB-T2::tInterLigths.0 = INTEGER: on(1)
MIB-T2::tInterLigths.0 = INTEGER: off(2)


MIB-T2::tACTemp.0 = INTEGER: 25
MIB-T2::tACTemp.0 = INTEGER: 25


In [45]:
%%bash

snmpgetnext -v2c -c private -M +. -m +MIB-T2 localhost tModelName tDoorIndex.3
snmpgetnext -v2c -c private -M +. -m +MIB-T2 localhost tModelName.0
snmpgetnext -v2c -c private -M +. -m +MIB-T2 localhost tDoorIndex.3
snmpgetnext -v2c -c private -M +. -m +MIB-T2 localhost tDoorIndex.4

MIB-T2::tModelName.0 = STRING: Tesla Model S 2024
MIB-T2::tDoorIndex.rear-left = INTEGER: rear-left(4)
MIB-T2::tManDate.0 = STRING: 2024-02-29 14:30:58
MIB-T2::tDoorIndex.rear-left = INTEGER: rear-left(4)
MIB-T2::tDoorStatus.front-right = INTEGER: open(1)
